In [14]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import folium
import branca
import panama_papers_aux
from IPython.display import Markdown

###### Versions

In [2]:
print('ipywidgets:', widgets.__version__)
print('folium:', folium.__version__)

ipywidgets: 7.0.0
folium: 0.5.0


# Simple queries

### Reading clean data

In [3]:
entities = pd.read_csv("./data/data_clean_csv/entities_clean.csv", dtype = 'object')
intermediaries = pd.read_csv("./data/data_clean_csv/intermediaries_clean.csv", dtype = 'object')
officers = pd.read_csv("./data/data_clean_csv/officers_clean.csv", dtype = 'object')
addresses = pd.read_csv("./data/data_clean_csv/addresses_clean.csv", dtype = 'object')
all_edges = pd.read_csv("./data/data_clean_csv/all_edges_clean.csv", dtype = 'object')

### DataFrame dictionary creation

We now create a dictionary that contains all the DataFrames:

In [4]:
df_dictionary = {'Entity': entities, 'Intermediary': intermediaries, 'Officer': officers, 'Address': addresses,
             'all_edges': all_edges}

### List of available countries

`available_countries` is a list containing all the countries that can be used as a filter for the query.

In [5]:
available_countries = [x for x in officers['countries'].unique() if ';' not in x]
available_countries.sort()
available_countries = ['All'] + available_countries
len(available_countries)

208

### GUI to get the relationships of a particular item

The `get_relationships` function is meant to find all the relationships of a particular item of the database, represented as DataFrame line that is passed as input (`df_queried_item`). It outputs a DataFrame showing all the in and out-relationships involving the queried item of the database.
This function is then used through a GUI interface at the end of this notebook.

In [7]:
panama_papers_aux.Item.readCapitalCoordinates()

In [17]:
options_type_dic = {'Entities': 'entities', 'Intermediaries': 'intermediaries', 'Officers': 'officers'}

option_list = ['Entity', 'Intermediary', 'Officer']
widgetDFName = widgets.Dropdown(description = 'Type', options = option_list)
widgetQueriedName = widgets.Text(description = 'Name', value = 'Type name here')
widgetCountry = widgets.Dropdown(description = 'Country', options = available_countries)
# 1st widget
first_widget = interactive(panama_papers_aux.search_by_name, queried_name = widgetQueriedName, dictionary = fixed(df_dictionary),\
                df_name = widgetDFName, country = widgetCountry)

# Button 1
button1 = widgets.Button(description="Apply Filter")
def show_results(b):
    widgetResult.options = first_widget.result['name'].values.tolist()
button1.on_click(show_results)

# 2nd widget
def name_selection(selected_name):
    matched_df = first_widget.result
    return matched_df[matched_df['name'] == selected_name]
widgetResult = widgets.Dropdown(description = 'Name options')
second_widget = interactive(name_selection, selected_name = widgetResult)

# Button 2
def print_selected_result(b):
    if (second_widget.result.shape[0] != 0):
        # Instantiating an Item object corresponding to the query
        # Then instantiating a NetworkItem object from the previous Item object
        type_ = widgetDFName.value
        queried_item = panama_papers_aux.Item.fromSingleLineDF(second_widget.result, type_)
        queried_item_network = panama_papers_aux.ItemNetwork(queried_item, df_dictionary)
        
        display()
        
        # get ItemNetwork DataFrame and print it
        network_df = queried_item_network.getDF()
        display()
        
        # get FoliumMap of connections and print it
        connection_map = queried_item_network.getMap()
        display(Markdown('### ' + queried_item.name), network_df, connection_map)
        #button2.close()
        #button3.open()

button2 = widgets.Button(description="Display Relationships")
button2.on_click(print_selected_result)

In [18]:
handle = display(first_widget, button1, second_widget, button2)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

### TRUMPCARD S.A.

,in_node,in_name,in_country,in_type,relationship,out_node,out_name,out_country,out_type
0,11010837,LATINCONSULT INC.,Panama,Intermediary,intermediary of,10005331,TRUMPCARD S.A.,Panama,Entity
